In [ ]:
import numpy as np
import pandas as pd
# import matplotlib
# matplotlib.use('TkAgg')
import math
import matplotlib.pyplot as plt

path ='/Users/jiayun/PycharmProjects/untitled/untitled/untitled/r' \
      'bot Project/ my code/machine learning/week3 logistic regression/ex2data1.txt'

data = pd.read_csv(path, header = None, names = ["Exam 1", "Exam 2", "Admitted"])
data.head()

'''1'''
##############################################################################
########---------------------plot 2 class part--------------------------########
##############################################################################
positive = data[data['Admitted'].isin([1])]
negative = data[data['Admitted'].isin([0])]

'''data['Admitted'].isin([1])   返回了一个false true 的矩阵， 外面再加一个data【】， 就实现了在这个dataframe里面提取出找到的'1'
另外一种写法：
## positive = data[data.iloc[:,2].isin([1])] ##
 这个和上面的命令一模一样， 只不过换了一个说法， data.iloc[:,1]
'''

# fig, ax = plt.subplots(figsize = (12,8))
# ax.scatter(positive['Exam 1'], positive['Exam 2'], s = 50 ,c = 'b', marker = 'o', label = 'Admitted')
# ax.scatter(negative['Exam 1'], negative['Exam 2'], s = 50, c = 'r', marker = 'x', label = 'Rejected')
# ax.legend( loc = 1 )   # 说明小图的位置， 默认是1，1 是右上角
# ax.set_xlabel('Exam 1 score')
# ax.set_ylabel('Exam 2 score')
# plt.show()

##############################################################################
########---------------------plot 2 class end--------------------------########
##############################################################################

'''2'''
##############################################################################
########---------------------def sigmoid --------------------------########
##############################################################################
def sigmoid(z):
      result = 1 / (1+ np.exp(-z))    # 这地方也可以用 np.exp(-z) 一样的
      return  result

'''测试这个sigmoid 能否正常工作'''
nums = np.arange(-10, 10, step=1)

fig, ax = plt.subplots(figsize = (12,8))  # 这个命令，画图必用。。哈哈哈
ax.plot(nums, sigmoid(nums), 'r')             # 就是 plot（x,y, 样式）
plt.show()

##############################################################################
########---------------------sigmoid end--------------------------########
##############################################################################

'''3'''
##############################################################################
########---------------------cost function --------------------------########
##############################################################################
def costFunction(theta, X, y):   # costfunction 就是 J（theta）
      theta = np.matrix(theta)
      X = np.matrix(X)
      y = np.matrix(y)

      first_term = np.multiply(-y , np.log(sigmoid(X * theta.T)))
      second_term = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
      jVal = np.sum(first_term - second_term) / (len(X))
      return jVal
'''下面是初始化， 初始化之后得到的结果是当theta = 【0，0，0】 时候的cost'''
##  初始化加计算部分
data.insert(0, 'ones', 1)
cols = data.shape[1]
features = cols-2

X = data.iloc[:,0:(cols-1)]
y = data.iloc[:,(cols-1):cols]

X = np.array(X.values)
y = np.array(y.values)
theta = np.zeros(cols-1)
costFunction(theta,X,y)
##############################################################################
########---------------------cost function end--------------------------########
##############################################################################

'''4'''
'''这是我自己写的梯度下降，效果不是很好， 因为logistic regression 的回归 梯度以及学习率不好掌握'''
##############################################################################
########---------------------gradient descent--------------------------########
##############################################################################
x = X
zero_vect = np.zeros(x.shape[1])
theta = np.matrix(zero_vect)

def gradient(theta,x,y,iter,alpha):
      x = np.matrix(x)
      y = np.matrix(y)
      parameters = int(theta.ravel().shape[1])
      p = parameters          # p 等于几 theta就有几个， 比如p=3， 就有 theta0， theta1，theta2
      m = x.shape[0]          # m 是样本个数

      temp = np.matrix(np.zeros(p))    # 初始化temp，让temp = 【0，0，0】， 最后的 temp 结果就是梯度下降后的 theta
      cost = np.zeros(iter)

      for i in range(iter):

            z = (x*(theta.T))
            error = sigmoid(z) - y

            for j in range(p):

                  term = np.multiply(error, x[:,j])
                  temp[0,j] = theta[0,j] - ((alpha * np.sum(term)) / m)

                  theta = temp
                  cost[i] =  costFunction(theta,x, y)

      return  theta, cost

alpha = 0.02
iter = 100
final_theta, final_cost_matrix = gradient(theta,x,y,iter,alpha)
final_cost = costFunction(final_theta,x,y)
t = final_theta
cm = final_cost_matrix
c = final_cost
print(t)
print(c)

## plot cost， 看看是不是从下降到converge
fig, ax = plt.subplots(figsize = (12,8))
ax.plot(np.arange(iter),final_cost_matrix,'r')   # np.arrange(iter) 是把1000重新arrange了， 变成了从 0 到 999
ax.set_xlabel('iterations')
ax.set_ylabel('Cost')
ax.set_title('Error VS Training')
plt.show()
''''''
##############################################################################
########---------------------gradient--------------------------########
##############################################################################

'''5'''

''' 用老师交的方法，  写function只包括求偏导的地方'''
##############################################################################
########---------------------def gradient--------------------------########
##############################################################################

zero_vect = np.zeros(X.shape[1])
    # 初始化theta

'''下面这个function 是梯度下降一步的计算，只下降了一次。'''
def gradient(theta,X, y):

      theta = np.matrix(theta)
      X = np.matrix(X)
      y = np.matrix(y)

      parameters = int(theta.ravel().shape[1])
      p = parameters
      m = X.shape[0]

      grad = np.zeros(p)            # grad 用来代替theta， 初始值是0
      error = sigmoid(X*theta.T) - y

      for j in range(p):
            term = np.multiply(error, X[:,j])
            grad[j] = np.sum(term) / m
      new_theta = grad
      return new_theta

grad = gradient(theta,X, y)

'''现在可以用SciPy's truncated newton（TNC）实现寻找最优参数'''
import scipy.optimize as opt

result = opt.fmin_tnc(func=costFunction, x0=theta, fprime=gradient, args=(X,y))
   # 这一步命令，func 和 fprime 一定要变量顺序一致， costFunction(theta,x, y), gradient(theta,x,y) 也得是 theta，x，y这个顺序，
 ## 要不然这个命令没法跑！

result      # 这个result是一个 元组
costFunction(result[0],X,y)
##############################################################################
########---------------------def gradient--------------------------########
##############################################################################

# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
#
# path ='/Users/jiayun/PycharmProjects/untitled/untitled/untitled/r' \
#       'bot Project/ my code/machine learning/week3 logistic regression/ex2data1.txt'
#
# data = pd.read_csv(path, header = None, names = ["Exam 1", "Exam 2", "Admitted"])
# data.head()
#
# positive = data[data['Admitted'].isin([1])]
# negative = data[data['Admitted'].isin([0])]
#
# fig, ax = plt.subplots(figsize=(12,8))
# ax.scatter(positive['Exam 1'], positive['Exam 2'], s=50, c='b', marker='o', label='Admitted')
# ax.scatter(negative['Exam 1'], negative['Exam 2'], s=50, c='r', marker='x', label='Not Admitted')
# ax.legend()
# ax.set_xlabel('Exam 1 Score')
# ax.set_ylabel('Exam 2 Score')
# plt.show()
#
#
# def  sigmoid(z):
#       return 1 / (1 + np.exp(-z))
#
# nums = np.arange(-10, 10, step=1)
#
# fig, ax = plt.subplots(figsize=(12,8))
# ax.plot(nums, sigmoid(nums), 'r')
# plt.show()
#
# def cost(theta, X, y):
#       theta = np.matrix(theta)
#       X = np.matrix(X)
#       y = np.matrix(y)
#       first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
#       second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
#       return np.sum(first - second) / (len(X))
#
#
#
# # add a ones column - this makes the matrix multiplication work out easier# add a
# data.insert(0, 'Ones', 1)
#
# # set X (training data) and y (target variable)
# cols = data.shape[1]
# X = data.iloc[:,0:cols-1]
# y = data.iloc[:,cols-1:cols]
#
# # convert to numpy arrays and initalize the parameter array theta
# X = np.array(X.values)
# y = np.array(y.values)
# theta = np.zeros(3)
#
#
# def gradient(theta, X, y):
#       theta = np.matrix(theta)
#       X = np.matrix(X)
#       y = np.matrix(y)
#
#       parameters = int(theta.ravel().shape[1])
#       grad = np.zeros(parameters)
#
#       error = sigmoid(X * theta.T) - y
#
#       for i in range(parameters):
#             term = np.multiply(error, X[:,i])
#             grad[i] = np.sum(term) / len(X)
#
#       return grad
#
# import scipy.optimize as opt
# result = opt.fmin_tnc(func=cost, x0=theta, fprime=gradient, args=(X, y))
# result
#
# cost(result[0], X, y)


'''6'''
'''prediction'''
##############################################################################
########---------------------def prediction --------------------------########
##############################################################################
def predict(theta, X):
      # theta = np.matrix(theta)
      # X = np.matrix(X)
      probability = sigmoid(X * theta.T)
      prediction = [ 1 if x >= 0.5 else 0 for x in probability ]   # prediction结果是一个list
      return prediction

'''上面的函数是求预测用的，下面求预测结果'''

'''下面这一段代码是关于匹配正确率的，仔细学习一下'''
theta_final = np.matrix(result[0])
y_prediction = predict(theta_final,X)
correct = [1 if((a==1 and b ==1) or (a==0 and b==0)) else 0 for (a,b) in zip(y_prediction, y) ]
accuracy = (sum(map(int, correct)) % len(correct))
print('accuracy = {0}%'.format(accuracy))
##############################################################################
########---------------------prediction end--------------------------########
##############################################################################